<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json

sys.path.insert(1, "../shared")  # add the shared directory to the Python path
import utils
import env_manager

# The deployment name is derived from the notebook file name.
# This is useful for organizing resources in Azure and keeping track of different deployments.
deployment_name = os.path.basename(os.path.dirname(globals()["__vsc_ipynb_file__"]))

resource_group_name = (
    f"lab-{deployment_name}"  # change the name to match your naming style
)
resource_group_location = "westeurope"

apim_resource_name = "apim"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"

# List of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_resources = [
    {"name": "openai1", "location": "swedencentral"},
    {"name": "openai2", "location": "swedencentral"},
]

openai_resources_sku = "S0"
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = "GlobalStandard"
openai_model_capacity = 20
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"
openai_specification_url = "https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/2024-02-01/inference.json"
openai_backend_pool = "openai-backend-pool"

log_analytics_name = "workspace"
app_insights_name = "insights"

function_app_name = "function"

storage_account_name = "storage"

# SQL Server and Database configuration
sql_server_name = "sqlserver"
sql_server_location = "westeurope"
sql_admin_login = "sqladmin"
sql_admin_password = "P@ssw0rd1234!"  # You should use a more secure method to handle passwords in production
sql_database_name = "salesdb"
sql_database_sku = "Basic"

# SQL Function App configuration
sql_function_app_name = "sqlfunction"
sql_function_app_location = "westeurope"

env_manager.update_notebook_env(
    {
        "resource_group_name": resource_group_name,
        "resource_group_location": resource_group_location,
        "apim_resource_name": apim_resource_name,
        "apim_resource_location": apim_resource_location,
        "apim_resource_sku": apim_resource_sku,
        "openai_resources": openai_resources,
        "openai_resources_sku": openai_resources_sku,
        "openai_model_name": openai_model_name,
        "openai_model_version": openai_model_version,
        "openai_model_sku": openai_model_sku,
        "openai_model_capacity": openai_model_capacity,
        "openai_deployment_name": openai_deployment_name,
        "openai_api_version": openai_api_version,
        "openai_specification_url": openai_specification_url,
        "openai_backend_pool": openai_backend_pool,
        "log_analytics_name": log_analytics_name,
        "app_insights_name": app_insights_name,
        "function_app_name": function_app_name,
        "storage_account_name": storage_account_name,
        "deployment_name": deployment_name,
        "sql_server_name": sql_server_name,
        "sql_server_location": sql_server_location,
        "sql_admin_login": sql_admin_login,
        "sql_admin_password": sql_admin_password,
        "sql_database_name": sql_database_name,
        "sql_database_sku": sql_database_sku,
        "sql_function_app_name": sql_function_app_name,
        "sql_function_app_location": sql_function_app_location,
    }
)

utils.print_ok("Notebook initialized")

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations.


In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIModelSKU": { "value": openai_model_sku },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name },
    "functionAppName": { "value": function_app_name },
    "storageAccountName": { "value": storage_account_name },
    "sqlServerName": { "value": sql_server_name },
    "sqlServerLocation": { "value": sql_server_location },
    "sqlAdministratorLogin": { "value": sql_admin_login },
    "sqlAdministratorLoginPassword": { "value": sql_admin_password },
    "sqlDatabaseName": { "value": sql_database_name },
    "sqlDatabaseSku": { "value": sql_database_sku },
    "sqlFunctionAppName": { "value": sql_function_app_name },
    "sqlFunctionAppLocation": { "value": sql_function_app_location }
  }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
import sys
sys.path.insert(1, "../shared")  # add the shared directory to the Python path
import env_manager

# Load all variables from previous notebooks (if any)
variables = env_manager.update_notebook_env()

# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)
    function_app_resource_name = utils.get_deployment_output(output, 'functionAppResourceName', 'Function App Resource Name')
    sql_server_fqdn = utils.get_deployment_output(output, 'sqlServerFqdn', 'SQL Server FQDN')
    sql_database_name = utils.get_deployment_output(output, 'sqlDatabaseName', 'SQL Database Name')
    sql_function_app_resource_name = utils.get_deployment_output(output, 'sqlFunctionAppResourceName', 'SQL Function App Resource Name')
    
    new_variables = {
        "apim_resource_gateway_url": apim_resource_gateway_url,
        "apim_subscription_key": apim_subscription_key,
        "function_app_resource_name": function_app_resource_name,
        "sql_server_fqdn": sql_server_fqdn,
        "sql_database_name": sql_database_name,
        "sql_function_app_resource_name": sql_function_app_resource_name,
        "sql_admin_login": sql_admin_login,
        "sql_admin_password": sql_admin_password,
    }
    env_manager.update_notebook_env(new_variables)

In [ ]:
import sys, os
sys.path.insert(1, "../shared")

function_app_resource_name = env_manager.get_variable_with_default("function_app_resource_name", globals())

os.chdir("functions/weather")
# Deploy the function app
try:
    os.system(f"func azure functionapp publish {function_app_resource_name}")
except Exception as e:
    print(f"Error deploying function app: {e}")
finally:
    os.chdir("../..")

<a id='function'></a>
### 🧪 Test the Weather Function

In [ ]:
import requests
import sys, json
sys.path.insert(1, "../shared")

apim_resource_gateway_url = env_manager.get_variable_with_default("apim_resource_gateway_url", globals())
apim_subscription_key = env_manager.get_variable_with_default("apim_subscription_key", globals())

request = { "location": "London", "unit": "celsius" }
url = apim_resource_gateway_url + "/weather"
response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = request)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("location: ", data.get("location"))
    print("unit: ", data.get("unit"))
    print("temperature: ", data.get("temperature"))
else:
    print(response.text)


<a id='5'></a>
### 5️⃣ Deploy the SQL Function

Deploy the SQL function app to execute SQL queries against the database

In [ ]:
import sys, os
sys.path.insert(1, "../shared")

sql_function_app_resource_name = env_manager.get_variable_with_default("sql_function_app_resource_name", globals())

os.chdir("functions/sql")
# Deploy the function app
try:
    os.system(f"func azure functionapp publish {sql_function_app_resource_name}")
except Exception as e:
    print(f"Error deploying function app: {e}")
finally:
    os.chdir("../..")

<a id='6'></a>
### 6️⃣ Populate the SQL Database

Create tables and populate the SQL database with sample data for the hackathon.

Run **sales_data.sql** in Azure SQL Database query editor to create the sales table and insert some data.

<a id='7'></a>
### 🧪 Test the SQL Function API

In [ ]:
import requests
import sys
import json
import time
sys.path.insert(1, "../shared")

# Give some time for the SQL function to deploy properly
time.sleep(10)

apim_resource_gateway_url = env_manager.get_variable_with_default("apim_resource_gateway_url", globals())
apim_subscription_key = env_manager.get_variable_with_default("apim_subscription_key", globals())

# Test a simple query
query_request = {"query": "SELECT TOP 5 * FROM vw_SalesSummary"}
url = apim_resource_gateway_url + "/sql"
response = requests.post(url, headers = {'api-key': apim_subscription_key}, json = query_request)

if response.status_code == 200:
    data = json.loads(response.text)
    print(f"Query returned {data.get('count')} records")
    print(f"Execution time: {data.get('executionTime')} seconds")
    print("\nSample data:")
    
    # Display the first few records in a formatted way
    results = data.get('results', [])
    if results:
        for i, record in enumerate(results[:3]):  # Show first 3 records
            print(f"\nRecord {i+1}:")
            for key, value in record.items():
                print(f"  {key}: {value}")
        
        if len(results) > 3:
            print("\n... more records available")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Test a more complex query with parameters
print("\n\nTesting a more complex query with parameters:")
query_request = {
    "query": "SELECT RegionName, SUM(TotalAmount) as TotalSales FROM vw_SalesSummary GROUP BY RegionName ORDER BY TotalSales DESC"
}

response = requests.post(url, headers = {'api-key': apim_subscription_key}, json = query_request)

if response.status_code == 200:
    data = json.loads(response.text)
    print(f"Query returned {data.get('count')} records")
    print(f"Execution time: {data.get('executionTime')} seconds")
    print("\nSales by Region:")
    
    results = data.get('results', [])
    if results:
        for record in results:
            print(f"{record['RegionName']}: ${record['TotalSales']:.2f}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)